In [1]:
#!pip3 install ray torch torchvision tabulate tensorboard
#!pip3 install 'ray[rllib]'
#!pip3 install ray

In [6]:
#!pip3 uninstall -y iglu && pip3 install git+https://github.com/iglu-contest/iglu.git
#!pip3 install 'ray[rllib]'
#!pip3 install wandb
#!pip3 install torch torchvision

In [1]:
import warnings
warnings.filterwarnings('ignore')

import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

from models import VisualEncoder
from train import *

In [2]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder(features_dim)
        #self.encoder.load_state_dict(
        #    torch.load("Visual Autoencoder weights and models/encoder_weigths.pth", map_location=torch.device('cpu'))
        #)
        self.action_head = nn.Linear(features_dim, action_space.n)
        self.value_head = nn.Linear(features_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [3]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [4]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=1000)
    env.update_taskset(TaskSet(preset=['C22']))
    env = PovOnlyWrapper(env)
    env = IgluActionWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 1,
             "sgd_minibatch_size": 256,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 1000,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO C22 not pretrained"
                  }
              }

        },
        loggers=[WandbLogger])#callbacks=[
        #    CustomLoggerCallback(),
        #])

2021-09-17 10:30:48,371	INFO wandb.py:170 -- Already logged into W&B.
2021-09-17 10:30:48,386	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


Trial name,status,loc
PPO_my_env_52b7f_00000,RUNNING,


wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
(pid=1260) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=1260)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=1260) 2021-09-17 10:30:52,488	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=1260) 2021-09-17 10:30:52,488	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=1256) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=1256)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=1260) 2021-09-17 10:30:59,280	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(pid=1256) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=1256)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=1256) /root/miniconda/envs/py37/lib/python3.7/site-pac

Result for PPO_my_env_52b7f_00000:
  agent_timesteps_total: 1000
  custom_metrics: {}
  date: 2021-09-17_10-32-01
  done: false
  episode_len_mean: 1000.0
  episode_media: {}
  episode_reward_max: -1.0
  episode_reward_mean: -1.0
  episode_reward_min: -1.0
  episodes_this_iter: 1
  episodes_total: 1
  experiment_id: 47168954e393400b9c4f2abc75550571
  hostname: linar-B360M-D2V
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.19999999999999996
          cur_lr: 5.000000000000001e-05
          entropy: 2.8840800762176513
          entropy_coeff: 0.009999999999999998
          kl: 0.0052462765865284805
          policy_loss: 0.10100602741456693
          total_loss: 0.3638703312828309
          vf_explained_var: -0.24777162075042725
          vf_loss: 0.29065584903906305
    num_agent_steps_sampled: 1000
    num_agent_steps_trained: 1000
    num_steps_sampled: 1000
    num_steps_trained: 1000
  iterations_since_restore: 1
  node_ip: 192.168.1.100


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_52b7f_00000,RUNNING,192.168.1.100:1260,190,2132.08,190000,0.01,1,0,995.9


Result for PPO_my_env_52b7f_00000:
  agent_timesteps_total: 191000
  custom_metrics: {}
  date: 2021-09-17_11-06-46
  done: false
  episode_len_mean: 995.9
  episode_media: {}
  episode_reward_max: 1.0
  episode_reward_mean: 0.01
  episode_reward_min: 0.0
  episodes_this_iter: 1
  episodes_total: 191
  experiment_id: 47168954e393400b9c4f2abc75550571
  hostname: linar-B360M-D2V
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.50625
          cur_lr: 5.000000000000001e-05
          entropy: 2.0665960788726805
          entropy_coeff: 0.009999999999999998
          kl: 0.010598838887983872
          policy_loss: -0.052793215851609905
          total_loss: -0.06752159482695991
          vf_explained_var: -0.6836352348327637
          vf_loss: 0.0005719186663756975
    num_agent_steps_sampled: 191000
    num_agent_steps_trained: 191000
    num_steps_sampled: 191000
    num_steps_trained: 191000
  iterations_since_restore: 191
  node_ip: 192.168.1.1

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_52b7f_00000,RUNNING,192.168.1.100:1260,191,2142.25,191000,0.01,1,0,995.9


Result for PPO_my_env_52b7f_00000:
  agent_timesteps_total: 192000
  custom_metrics: {}
  date: 2021-09-17_11-06-56
  done: false
  episode_len_mean: 995.9
  episode_media: {}
  episode_reward_max: 1.0
  episode_reward_mean: 0.01
  episode_reward_min: 0.0
  episodes_this_iter: 1
  episodes_total: 192
  experiment_id: 47168954e393400b9c4f2abc75550571
  hostname: linar-B360M-D2V
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.50625
          cur_lr: 5.000000000000001e-05
          entropy: 2.1529735565185546
          entropy_coeff: 0.009999999999999998
          kl: 0.012683575204726181
          policy_loss: -0.027272874825737543
          total_loss: -0.041187533599117566
          vf_explained_var: -0.4902389645576477
          vf_loss: 0.0011940196690071994
    num_agent_steps_sampled: 192000
    num_agent_steps_trained: 192000
    num_steps_sampled: 192000
    num_steps_trained: 192000
  iterations_since_restore: 192
  node_ip: 192.168.1.

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_52b7f_00000,RUNNING,192.168.1.100:1260,192,2152.57,192000,0.01,1,0,995.9
